# SQL Generation with Transformer API

In [ ]:
!pip install torch transformers bitsandbytes accelerate sqlparse

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 49.0 MB/s eta 0:00:0000:010:01


In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

In [2]:
import torch

# Verificar si CUDA está disponible
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Usando GPU:", torch.cuda.get_device_name(device))
else:
    device = torch.device("cpu")
    print("Usando CPU")

Usando GPU: Tesla T4


In [3]:
available_memory = torch.cuda.get_device_properties(0).total_memory

In [4]:
print(available_memory)

15828320256


##Download the Model
Use any model on Colab (or any system with >30GB VRAM on your own machine) to load this in f16. If unavailable, use a GPU with minimum 8GB VRAM to load this in 8bit, or with minimum 5GB of VRAM to load in 4bit.

This step can take around 5 minutes the first time. So please be patient :)

In [5]:
model_name = "defog/sqlcoder-7b-2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
if available_memory > 15e9:
    # if you have atleast 15GB of GPU memory, run load the model in float16
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        trust_remote_code=True,
        torch_dtype=torch.float16,
        device_map="auto",
        use_cache=True,
    )
else:
    # else, load in 8 bits – this is a bit slower
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        trust_remote_code=True,
        # torch_dtype=torch.float16,
        load_in_8bit=True,
        device_map="auto",
        use_cache=True,
    )

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.84k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/515 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/3.59G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

##Set the Question & Prompt and Tokenize
Feel free to change the schema in the prompt below to your own schema

In [6]:
prompt = """### Task
Generate a SQL query to answer [QUESTION]{question}[/QUESTION]

### Instructions
- If you cannot answer the question with the available database schema, return 'I do not know'
- Remember that revenue is price multiplied by quantity
- Remember that cost is supply_price multiplied by quantity

### Database Schema
This query will run on a database whose schema is represented in this string:
CREATE TABLE products (
  product_id INTEGER PRIMARY KEY, -- Unique ID for each product
  name VARCHAR(50), -- Name of the product
  price DECIMAL(10,2), -- Price of each unit of the product
  quantity INTEGER  -- Current quantity in stock
);

CREATE TABLE customers (
   customer_id INTEGER PRIMARY KEY, -- Unique ID for each customer
   name VARCHAR(50), -- Name of the customer
   address VARCHAR(100) -- Mailing address of the customer
);

CREATE TABLE salespeople (
  salesperson_id INTEGER PRIMARY KEY, -- Unique ID for each salesperson
  name VARCHAR(50), -- Name of the salesperson
  region VARCHAR(50) -- Geographic sales region
);

CREATE TABLE sales (
  sale_id INTEGER PRIMARY KEY, -- Unique ID for each sale
  product_id INTEGER, -- ID of product sold
  customer_id INTEGER,  -- ID of customer who made purchase
  salesperson_id INTEGER, -- ID of salesperson who made the sale
  sale_date DATE, -- Date the sale occurred
  quantity INTEGER -- Quantity of product sold
);

CREATE TABLE product_suppliers (
  supplier_id INTEGER PRIMARY KEY, -- Unique ID for each supplier
  product_id INTEGER, -- Product ID supplied
  supply_price DECIMAL(10,2) -- Unit price charged by supplier
);

-- sales.product_id can be joined with products.product_id
-- sales.customer_id can be joined with customers.customer_id
-- sales.salesperson_id can be joined with salespeople.salesperson_id
-- product_suppliers.product_id can be joined with products.product_id

### Answer
Given the database schema, here is the SQL query that answers [QUESTION]{question}[/QUESTION]
[SQL]
"""

##Generate the SQL
This can be excruciatingly slow on a T4 in Colab, and can take 10-20 seconds per query. On faster GPUs, this will take ~1-2 seconds

Ideally, you should use `num_beams`=4 for best results. But because of memory constraints, we will stick to just 1 for now.

In [7]:
import sqlparse

def generate_query(question):
    updated_prompt = prompt.format(question=question)
    inputs = tokenizer(updated_prompt, return_tensors="pt").to("cuda")
    generated_ids = model.generate(
        **inputs,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
        max_new_tokens=400,
        do_sample=False,
        num_beams=1,
    )
    outputs = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

    torch.cuda.empty_cache()
    torch.cuda.synchronize()
    # empty cache so that you do generate more results w/o memory crashing
    # particularly important on Colab – memory management is much more straightforward
    # when running on an inference service
    return sqlparse.format(outputs[0].split("[SQL]")[-1], reindent=True)

In [8]:
question = "What was our revenue by product in the New York region last month?"
generated_sql = generate_query(question)

In [9]:
print(generated_sql)


SELECT p.product_id,
       SUM(s.quantity * p.price) AS revenue
FROM sales s
JOIN salespeople sp ON s.salesperson_id = sp.salesperson_id
JOIN products p ON s.product_id = p.product_id
WHERE sp.region = 'New York'
  AND s.sale_date >= (CURRENT_DATE - INTERVAL '1 month')
GROUP BY p.product_id
ORDER BY revenue DESC NULLS LAST;


# Exercise
 - Complete the prompts similar to what we did in class.
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

In [ ]:
prompt1 = """### Task
Generate a SQL query that answers the following question: [QUESTION]{question}[/QUESTION]

### Instructions
- If you cannot answer the question using the provided database schema, return 'I do not know'.
- Compute revenue as price multiplied by quantity.
- Compute cost as supply_price multiplied by quantity.

### Database Schema
This query will be executed on a database with the following schema:
CREATE TABLE products (
  product_id INTEGER PRIMARY KEY, -- Unique product identifier
  name VARCHAR(50), -- Name of the product
  price DECIMAL(10,2), -- Unit price
  quantity INTEGER  -- Stock quantity
);

CREATE TABLE customers (
  customer_id INTEGER PRIMARY KEY, -- Unique customer identifier
  name VARCHAR(50), -- Customer name
  address VARCHAR(100) -- Mailing address
);

CREATE TABLE salespeople (
  salesperson_id INTEGER PRIMARY KEY, -- Unique salesperson identifier
  name VARCHAR(50), -- Salesperson name
  region VARCHAR(50) -- Sales region
);

CREATE TABLE sales (
  sale_id INTEGER PRIMARY KEY, -- Unique sale identifier
  product_id INTEGER, -- ID of the product sold
  customer_id INTEGER,  -- ID of the purchasing customer
  salesperson_id INTEGER, -- ID of the salesperson
  sale_date DATE, -- Date of sale
  quantity INTEGER -- Number of units sold
);

CREATE TABLE product_suppliers (
  supplier_id INTEGER PRIMARY KEY, -- Unique supplier identifier
  product_id INTEGER, -- Product ID supplied
  supply_price DECIMAL(10,2) -- Supplier's unit price
);

-- Join conditions:
-- sales.product_id joins with products.product_id
-- sales.customer_id joins with customers.customer_id
-- sales.salesperson_id joins with salespeople.salesperson_id
-- product_suppliers.product_id joins with products.product_id

### Answer
Based on the above schema, please provide the SQL query that answers [QUESTION]{question}[/QUESTION]
[SQL]
"""


In [ ]:
prompt2 = """### Task
Construct a SQL query to address the following question: [QUESTION]{question}[/QUESTION]

### Instructions
- If the provided database schema does not allow you to answer the question, output 'I do not know'.
- Note that revenue is defined as price multiplied by quantity.
- Note that cost is defined as supply_price multiplied by quantity.

### Database Schema
The SQL query will run on a database with this schema:
CREATE TABLE products (
  product_id INTEGER PRIMARY KEY, -- Unique ID for each product
  name VARCHAR(50), -- Product name
  price DECIMAL(10,2), -- Price per unit
  quantity INTEGER  -- Current inventory
);

CREATE TABLE customers (
  customer_id INTEGER PRIMARY KEY, -- Unique ID for each customer
  name VARCHAR(50), -- Customer's name
  address VARCHAR(100) -- Customer's address
);

CREATE TABLE salespeople (
  salesperson_id INTEGER PRIMARY KEY, -- Unique ID for each salesperson
  name VARCHAR(50), -- Salesperson's name
  region VARCHAR(50) -- Sales region
);

CREATE TABLE sales (
  sale_id INTEGER PRIMARY KEY, -- Unique sale ID
  product_id INTEGER, -- ID of the sold product
  customer_id INTEGER,  -- ID of the customer making the purchase
  salesperson_id INTEGER, -- ID of the salesperson handling the sale
  sale_date DATE, -- Date when the sale occurred
  quantity INTEGER -- Quantity sold
);

CREATE TABLE product_suppliers (
  supplier_id INTEGER PRIMARY KEY, -- Unique supplier ID
  product_id INTEGER, -- Supplied product ID
  supply_price DECIMAL(10,2) -- Unit cost from the supplier
);

-- Relationships:
-- sales.product_id -> products.product_id
-- sales.customer_id -> customers.customer_id
-- sales.salesperson_id -> salespeople.salesperson_id
-- product_suppliers.product_id -> products.product_id

### Answer
Here is the SQL query based on the above schema that answers [QUESTION]{question}[/QUESTION]
[SQL]
"""


In [ ]:
prompt3 = """### Task
Write a SQL query to answer the following question: [QUESTION]{question}[/QUESTION]

### Guidelines
- If you are unable to derive an answer from the given database schema, simply return 'I do not know'.
- Remember: revenue equals price times quantity.
- Remember: cost equals supply_price times quantity.

### Database Schema
The query should be executed on a database with the following structure:
CREATE TABLE products (
  product_id INTEGER PRIMARY KEY, -- Unique product ID
  name VARCHAR(50), -- Name of the product
  price DECIMAL(10,2), -- Price per unit
  quantity INTEGER  -- Available stock
);

CREATE TABLE customers (
  customer_id INTEGER PRIMARY KEY, -- Unique customer ID
  name VARCHAR(50), -- Customer name
  address VARCHAR(100) -- Customer address
);

CREATE TABLE salespeople (
  salesperson_id INTEGER PRIMARY KEY, -- Unique salesperson ID
  name VARCHAR(50), -- Salesperson name
  region VARCHAR(50) -- Geographic region
);

CREATE TABLE sales (
  sale_id INTEGER PRIMARY KEY, -- Unique sale ID
  product_id INTEGER, -- Product ID sold
  customer_id INTEGER,  -- Customer ID
  salesperson_id INTEGER, -- Salesperson ID
  sale_date DATE, -- Date of sale
  quantity INTEGER -- Quantity sold
);

CREATE TABLE product_suppliers (
  supplier_id INTEGER PRIMARY KEY, -- Unique supplier ID
  product_id INTEGER, -- Product ID supplied
  supply_price DECIMAL(10,2) -- Supplier's price per unit
);

-- Note the following join relationships:
-- sales.product_id matches products.product_id
-- sales.customer_id matches customers.customer_id
-- sales.salesperson_id matches salespeople.salesperson_id
-- product_suppliers.product_id matches products.product_id

### Answer
Below is the SQL query that answers the question based on the provided schema: [QUESTION]{question}[/QUESTION]
[SQL]
"""
